In [1]:
!wget https://jdbc.postgresql.org/download/postgresql-42.7.3.jar

--2025-09-01 22:48:45--  https://jdbc.postgresql.org/download/postgresql-42.7.3.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1089312 (1.0M) [application/java-archive]
Saving to: ‘postgresql-42.7.3.jar’

postgresql-42.7.3.j 100%[===================>]   1.04M  6.06MB/s    in 0.2s    

2025-09-01 22:48:45 (6.06 MB/s) - ‘postgresql-42.7.3.jar’ saved [1089312/1089312]



# Загрузка спарсенных данных в PySpark

In [ ]:
# creds
POSTGRES_CONFIG = {
    'host': 'my_host',
    'port': 6432,
    'user': 'my_user',
    'password': 'my_password',
    'database': 'my_datebase'
}

jdbc_url = f"jdbc:postgresql://{POSTGRES_CONFIG['host']}:{POSTGRES_CONFIG['port']}/{POSTGRES_CONFIG['database']}"
print(f"JDBC URL: {jdbc_url}")

In [3]:
# Создание Spark сессии
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("PostgresConnection") \
    .config("spark.jars", "/content/postgresql-42.7.3.jar") \
    .getOrCreate()

In [4]:
# чтение сырых данных из таблицы "public.zolotoy_raw_products", куда производился парсинг

df = spark.read.format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "public.zolotoy_raw_products") \
    .option("user", POSTGRES_CONFIG['user']) \
    .option("password", POSTGRES_CONFIG['password']) \
    .option("driver", "org.postgresql.Driver") \
    .load()

df.show(20, truncate=False)

+-------+--------------------------+-------------------------+---------------------------------------------------+--------+---------+--------+------+-------+---------------------------------------------------+-----------+
|sku    |category                  |subcategory              |name                                               |price   |old_price|discount|rating|reviews|product_url                                        |parsed_date|
+-------+--------------------------+-------------------------+---------------------------------------------------+--------+---------+--------+------+-------+---------------------------------------------------+-----------+
|1978724|Кольца                    |Женские кольца           |Золотое кольцо с сапфиром выращенным и бриллиантами|12 790 ₽|25 580 ₽ |−50%    |235   |44     |https://www.585zolotoy.ru/catalog/products/1978724/|2025-08-31 |
|9001102|Кольца                    |Мужские кольца           |Золотое обручальное кольцо                        

In [5]:
print(f"Количество строк в исходной таблице public.zolotoy_raw_products: {df.count()}")

Количество строк в исходной таблице public.zolotoy_raw_products: 9931


In [6]:
# мой список парсинных категорий

parsed_category = ["Кольца"]
parsed_category

['Кольца']

# Разведочный анализ данных


## Анализ и восстановление поля category

In [7]:
# Уникальные значения category

df.select("category").distinct().show(truncate=False)

+--------------------------+
|category                  |
+--------------------------+
|Кольца                    |
|Подвески                  |
|Религия                   |
|Тренды ювелирных украшений|
|NULL                      |
+--------------------------+



In [8]:
# отбор невалидных категорий, т.е тех, которые не в списке или с NULL значением

df.filter((F.col("category").isNull()) | (~F.col("category").isin(parsed_category))).show(truncate=False)

+-------+--------------------------+----------------------+------------------------------------------------------+--------+---------+--------+------+-------+---------------------------------------------------+-----------+
|sku    |category                  |subcategory           |name                                                  |price   |old_price|discount|rating|reviews|product_url                                        |parsed_date|
+-------+--------------------------+----------------------+------------------------------------------------------+--------+---------+--------+------+-------+---------------------------------------------------+-----------+
|9002261|Тренды ювелирных украшений|Ювелирная база        |Золотое кольцо с премиум цирконием                    |13 990 ₽|27 980 ₽ |−50%    |125   |22     |https://www.585zolotoy.ru/catalog/products/9002261/|2025-08-31 |
|1847403|Тренды ювелирных украшений|Ювелирная база        |Золотое кольцо с бриллиантом                         

In [9]:
# попытка восстановить категорию и подкатегорию из ключевых слов в названии:
# category = "Кольца" если name = "%кольцо%" или name = "%печатка%"

df = df.withColumn("category",
    F.when(F.lower(F.col("name")).like("%кольцо%"), "Кольца")
     .when(F.lower(F.col("name")).like("%печатка%"), "Кольца")
     .otherwise(F.col("category"))
)

df.filter((F.col("category").isNull()) | (~F.col("category").isin(parsed_category))).show(20, truncate=False)

+-------+--------+----------------+---------------------------------------+--------+---------+--------+------+-------+---------------------------------------------------+-----------+
|sku    |category|subcategory     |name                                   |price   |old_price|discount|rating|reviews|product_url                                        |parsed_date|
+-------+--------+----------------+---------------------------------------+--------+---------+--------+------+-------+---------------------------------------------------+-----------+
|9002563|Подвески|Золотые подвески|Золотая подвеска с цитрином и фианитами|17 095 ₽|75 980 ₽ |−78%    |20    |1      |https://www.585zolotoy.ru/catalog/products/9002563/|2025-08-31 |
+-------+--------+----------------+---------------------------------------+--------+---------+--------+------+-------+---------------------------------------------------+-----------+



## Анализ поля subcategory

In [10]:
# отбор невалидных подкатегорий и с NULL значениями

df.filter((F.col("subcategory").isNull())).show(truncate=False)

+-------+--------+-----------+-----------------------------+--------+---------+--------+------+-------+---------------------------------------------------+-----------+
|sku    |category|subcategory|name                         |price   |old_price|discount|rating|reviews|product_url                                        |parsed_date|
+-------+--------+-----------+-----------------------------+--------+---------+--------+------+-------+---------------------------------------------------+-----------+
|7209694|Кольца  |NULL       |Золотое обручальное кольцо   |42 834 ₽|155 760 ₽|−72%    |265   |52     |https://www.585zolotoy.ru/catalog/products/7209694/|2025-08-31 |
|2792739|Кольца  |NULL       |Золотое обручальное кольцо   |31 240 ₽|62 470 ₽ |−50%    |215   |40     |https://www.585zolotoy.ru/catalog/products/2792739/|2025-08-31 |
|4025313|Кольца  |NULL       |Золотое обручальное кольцо   |29 381 ₽|106 830 ₽|−72%    |265   |58     |https://www.585zolotoy.ru/catalog/products/4025313/|2025-

Однозначно, по ключевому слову в имени, восстановить поддкатегорию не удасться

## Анализ поля price и old_price

In [11]:
# Последний символ каждой строки price и находим уникальные

(
    df.select(
    F.substring(F.col("price"), -1, 1).alias("last_symbol"))
    .distinct().show()
)

+-----------+
|last_symbol|
+-----------+
|          ₽|
|       NULL|
+-----------+



In [12]:
print("Все строки содержащие отрицательные значения цен")

df.filter(
    (F.col('price') < 0) |
    (F.col('old_price') < 0)
).show(truncate=False)

Все строки содержащие отрицательные значения цен
+---+--------+-----------+----+-----+---------+--------+------+-------+-----------+-----------+
|sku|category|subcategory|name|price|old_price|discount|rating|reviews|product_url|parsed_date|
+---+--------+-----------+----+-----+---------+--------+------+-------+-----------+-----------+
+---+--------+-----------+----+-----+---------+--------+------+-------+-----------+-----------+



Есть товары без цены (недоступные для заказа), без указания скидки, без указания старой цены

<!-- Есть товары с ценой NULL (недоступные для заказа), без указания старой цены без скидки -->

# 3. Преобразование и очистка данных

In [13]:
# удаление | в конце

df = df.withColumn(
    "product_url",
    F.regexp_replace(F.col("product_url"), "%7C$", "")
)

In [14]:
# Удаляем все не-цифровые символы

df = df.withColumn("price",
                  F.regexp_replace(F.col("price"), r'[^\d]', '').cast('int')) \
       .withColumn("old_price",
                  F.regexp_replace(F.col("old_price"), r'[^\d]', '').cast('int'))

In [15]:
# очистка и преобразование скидки

def clean_discount(column):
    return F.when(column.isNotNull(),
                 F.abs(  # значение по модулю
                     F.replace(
                         F.replace(column, F.lit("−"), F.lit("-")),
                         F.lit("%"), F.lit("")
                     ).cast("int")
                 )
         ).otherwise(F.lit(None))

df = df.withColumn("discount", clean_discount(F.col("discount")))

In [16]:
# очистка и преобразование просмотров

def clean_reviews(column):
    return F.when(column.isNotNull(),
                 F.replace(column, F.lit("Без"), F.lit("0")).cast("int")
         ).otherwise(F.lit(None))

df = df.withColumn("reviews", clean_reviews(F.col("reviews")))

In [17]:
# просмотр

df.select('price', 'old_price', 'discount', 'reviews').show(5, truncate=False)

+-----+---------+--------+-------+
|price|old_price|discount|reviews|
+-----+---------+--------+-------+
|12790|25580    |50      |44     |
|12083|43930    |72      |48     |
|41495|165980   |75      |31     |
|13990|27980    |50      |22     |
|9990 |NULL     |NULL    |237    |
+-----+---------+--------+-------+
only showing top 5 rows



In [18]:
print("Количество null значений после восстановления:")

df.select([F.sum(F.isnull(c).cast("int")).alias(c) for c in df.columns]).show()

Количество null значений после восстановления:
+---+--------+-----------+----+-----+---------+--------+------+-------+-----------+-----------+
|sku|category|subcategory|name|price|old_price|discount|rating|reviews|product_url|parsed_date|
+---+--------+-----------+----+-----+---------+--------+------+-------+-----------+-----------+
|  0|       0|         36|   0| 2051|     2059|    2059|     0|      0|          0|          0|
+---+--------+-----------+----+-----+---------+--------+------+-------+-----------+-----------+



Вроде норм, можно грузить в  базу

# Запись очищенных данных в БД

In [19]:
# Преобразование типов данных

df_clean = df.withColumn("sku", F.col("sku").cast("int")) \
             .withColumn("price", F.col("price").cast("int")) \
             .withColumn("old_price", F.col("old_price").cast("int")) \
             .withColumn("discount", F.col("discount").cast("int")) \
             .withColumn("rating", F.col("rating").cast("int")) \
             .withColumn("reviews", F.col("reviews").cast("int")) \
             .withColumn("product_url", F.col("product_url").cast("string")) \
             .withColumn("parsed_date", F.col("parsed_date").cast("date")) \
             .withColumn("inserted_date", F.date_format(F.current_timestamp(), "yyyy-MM-dd HH:mm:ss"))

In [20]:
# Просто записываем как таблицу - Spark сам создаст её

# Запись в PostgreSQL
df_clean.write.format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "public.zolotoy_clean_products") \
    .option("user", POSTGRES_CONFIG['user']) \
    .option("password", POSTGRES_CONFIG['password']) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("Данные успешно загружены в zolotoy_clean_products")

Данные успешно загружены в zolotoy_clean_products


In [21]:
# Чтение очищенных данных из таблицы "public.zolotoy_clean_products"
df_clean = spark.read.format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "public.zolotoy_clean_products") \
    .option("user", POSTGRES_CONFIG['user']) \
    .option("password", POSTGRES_CONFIG['password']) \
    .option("driver", "org.postgresql.Driver") \
    .load()

# Показываем данные
df_clean.show(20, truncate=False)

+-------+--------+-------------------------+---------------------------------------------------+-----+---------+--------+------+-------+---------------------------------------------------+-----------+-------------------+
|sku    |category|subcategory              |name                                               |price|old_price|discount|rating|reviews|product_url                                        |parsed_date|inserted_date      |
+-------+--------+-------------------------+---------------------------------------------------+-----+---------+--------+------+-------+---------------------------------------------------+-----------+-------------------+
|1978724|Кольца  |Женские кольца           |Золотое кольцо с сапфиром выращенным и бриллиантами|12790|25580    |50      |235   |44     |https://www.585zolotoy.ru/catalog/products/1978724/|2025-08-31 |2025-09-01 22:49:34|
|9001102|Кольца  |Мужские кольца           |Золотое обручальное кольцо                         |12083|43930    |72  

In [22]:
print(f"Количество строк в очищенной таблице public.zolotoy_clean_products: {df_clean.count()}")

Количество строк в очищенной таблице public.zolotoy_clean_products: 9931


In [ ]:
print("Количество товаров по категориям:")
df_clean.groupBy("category").count().orderBy("count", ascending=False).show(truncate=False)

📈 Количество товаров по категориям:
+--------+-----+
|category|count|
+--------+-----+
|Кольца  |9930 |
|Подвески|1    |
+--------+-----+



**В БД product_url и parsed_date отбражаются корректно**

In [24]:
# Останавливаем SparkSession
spark.stop()